In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import numpy as np
import scipy.stats
import config as cfg
import os

N_BOOT = 3
N_REPEATS = 1
NEW_DATASET = False
DATASET = "xjtu"
TEST_SIZE= 0.3
TYPE= "bootstrap" # not_correlated
LINE_PLOT= 3
FEATURE_TO_SPLIT= "rms"
SPLIT_THRESHOLD= []
SPLITTED = True

In [2]:
# Load results into cv results
cv_results = pd.DataFrame()
dataset_names = ["xjtu"]
type_names = ["bootstrap"]
model_names = ["CoxPH", "RSF", "CoxBoost", "DeepSurv", "DSM", "WeibullAFT"]

for dataset_name in dataset_names:
    for type_name in type_names:
        for model_name in model_names:
            logs_dir = os.path.join(os.getcwd(), f"../data/logs/{dataset_name}/{type_name}")
            model_results = pd.read_csv(os.path.join(logs_dir, f'{model_name}_results.csv'))
            model_results["DatasetName"] = dataset_name
            model_results["TypeName"] = type_name
            cv_results = pd.concat([cv_results, model_results], axis=0)
            cv_results = cv_results.dropna().reset_index(drop=True)

In [3]:
# Print results
for dataset_name in dataset_names:
    for type_name in type_names:
        for model_name in model_names:
            text = ""
            results = cv_results.loc[(cv_results['DatasetName'] == dataset_name)
                                    & (cv_results['TypeName'] == type_name)
                                    & (cv_results['ModelName'] == model_name)]
            text += f"{model_name}"
            for ft_selector in ["NoneSelector", "PHSelector"]:
                ci_mean = results.loc[results['FtSelectorName'] == ft_selector]["CIndex"].mean().round(2)
                ci_std = results.loc[results['FtSelectorName'] == ft_selector]["CIndex"].std().round(2)
                ibs_mean = results.loc[results['FtSelectorName'] == ft_selector]["BrierScore"].mean().round(2)
                ibs_std = results.loc[results['FtSelectorName'] == ft_selector]["BrierScore"].std().round(2)
                text += f" & {ci_mean} ($\pm$ {ci_std}) & {ibs_mean} ($\pm$ {ibs_std})"
            text += " \\\\"
            print(text)
        print()

CoxPH & 0.56 ($\pm$ 0.07) & 0.17 ($\pm$ 0.04) & 0.56 ($\pm$ 0.07) & 0.17 ($\pm$ 0.04) \\
RSF & 0.81 ($\pm$ 0.02) & 0.06 ($\pm$ 0.03) & 0.82 ($\pm$ 0.02) & 0.06 ($\pm$ 0.03) \\
CoxBoost & 0.79 ($\pm$ 0.13) & 0.08 ($\pm$ 0.05) & 0.79 ($\pm$ 0.13) & 0.08 ($\pm$ 0.04) \\
DeepSurv & 0.47 ($\pm$ 0.14) & 0.16 ($\pm$ 0.03) & 0.55 ($\pm$ 0.08) & 0.16 ($\pm$ 0.03) \\
DSM & 0.59 ($\pm$ 0.03) & 0.19 ($\pm$ 0.04) & 0.67 ($\pm$ 0.07) & 0.19 ($\pm$ 0.02) \\
WeibullAFT & 0.57 ($\pm$ 0.03) & 0.17 ($\pm$ 0.02) & 0.57 ($\pm$ 0.03) & 0.17 ($\pm$ 0.02) \\



In [8]:
import numpy as np
import scipy.stats

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

ci = cv_results.loc[(cv_results['ModelName'] == 'CoxPH')]['CIndex']
ibs = cv_results.loc[(cv_results['ModelName'] == 'CoxPH')]['BrierScore']

print(mean_confidence_interval(ci))
print(mean_confidence_interval(ibs))

(0.5595157603271194, 0.49398460048983794, 0.6250469201644008)
(0.16741996656773325, 0.13432129160144501, 0.2005186415340215)
